# Introduction to Events

Events are the fundamental data object in Sentenai. In other words, everything else depends on Events. So what are events? Events are information about something that happened at some point in time. Defined more formally, an event:
1. Has a timestamp representing the moment in time when the event started happening
2. Has information associated with that event (we call this `data`)
3. May have a duration
4. May have a unique id tied to a particular collection, or stream, of events

## Defining an Event

Let's start by defining some events and looking at their properties.

In [2]:
from sentenai import Event
from datetime import datetime, timedelta
event0 = Event()
event1 = Event(ts=datetime(2010,1,1), data={'a': 5})
event2 = Event(ts=datetime(2011,1,1), data={'a': 7})
event3 = Event(ts=datetime(2010, 12, 2), duration=timedelta(days=45), data={'a': 3, 'b': {'c': True}})
event4 = Event(ts=datetime(2010, 12, 2), duration=timedelta(days=3), data={'a': 3, 'b': {'c': False}})


Let's start by looking at `event0`. It's an empty event. No id, no timestamp, no data.

In [14]:
event0

Event()

Why is this a valid event? Because `duration` and `data` are optional, and `ts` and `id` can be automatically assigned when the event is added to a stream. In short, events only require a timestamp and id to be assigned when they are added to an ordered collection of events (an event stream). Just creating a new Event object in Python doesn't save it anywhere. It's only when assigned to a stream that this event is persisted. This allows us to build up events incrementally before storing them on the backend.

### Ordering and equality: timestamp, then duration
It can be useful to compare the ordering of two events in time, so the default behavior of comparison operations between events is to compare timestamp, then duration.

In [16]:
sorted([event1,event2,event3,event4])

[Event(ts=numpy.datetime64('2010-01-01T00:00:00.000000'), data={'a': 5}),
 Event(ts=numpy.datetime64('2010-12-02T00:00:00.000000'), duration=numpy.timedelta64(259200000000,'us'), data={'a': 3, 'b': {'c': False}}),
 Event(ts=numpy.datetime64('2010-12-02T00:00:00.000000'), duration=numpy.timedelta64(3888000000000,'us'), data={'a': 3, 'b': {'c': True}}),
 Event(ts=numpy.datetime64('2011-01-01T00:00:00.000000'), data={'a': 7})]

Events are considered equal if their timestamp and duration is the same. This does not mean events have identical data or ids. To check whether an event has the same id or same data, check directly on those properties:

In [19]:
eq1 = Event(id="33AG", ts=datetime(2015,1,1,1), duration=timedelta(seconds=30), data={'a': 5, 'b': 6})
eq2 = Event(id="33AG", ts=datetime(2015,1,1,1), duration=timedelta(seconds=30), data={'a': 3, 'b': 6})
eq3 = Event(id="23AF", ts=datetime(2015,1,1,1), duration=timedelta(seconds=30), data={'a': 5, 'b': 6})


print("Events are equal?", "\t1 & 2: {}".format(eq1 == eq2), "\t2 & 3: {}".format(eq1 == eq3), sep='\n')
print("Event ids are equal?", "\t1 & 2: {}".format(eq1.id == eq2.id), "\t2 & 3: {}".format(eq1.id == eq3.id), sep='\n')
print("Event data are equal?", "\t1 & 2: {}".format(eq1.data == eq2.data), "\t2 & 3: {}".format(eq1.data == eq3.data), sep='\n')

Events are equal?
	1 & 2: True
	2 & 3: True
Event ids are equal?
	1 & 2: True
	2 & 3: False
Event data are equal?
	1 & 2: False
	2 & 3: True


### Changing Events

Editing the properties of an event is supported, but will *not* result in any API calls to Sentenai. To persist a changed event, you must associate the update with a particular stream.

### Two Kinds of Time

Events support two kinds of timestamps to handle cases of events that happen in both "real time" and "virtual time".

The first kind of timestamp is the _datetime object_, which represents "real time" events, where the event corresponds to a known point in universal time. Events with real timestamps must be inserted into streams which are anchored to real time, with a point, called `t0`, representing the start of that stream's *epoch*.

The second kind of timestamp is not a timestamp at all but a _signed integer_ representation of the number of nanoseconds since `0` that an event corresponds to in "virtual timee". These kinds of streams are useful for recording data where current time is not known and must be associated afterwards, as well as for recording data that has no basis in "real time", like a planned path.

In [5]:
virtual_time_event = Event(ts=55)
real_time_event = Event(ts=datetime(2015,1,1))

print(virtual_time_event, real_time_event, sep='\n')

Event(ts=55)
Event(ts=numpy.datetime64('2015-01-01T00:00:00.000000'))


### [Next chapter: Streams](Streams.ipynb)